# 在Cython模块之间共享函数

## 使用pxd文件声明输出的函数

In [1]:
%load_ext cython

当存在与`pyx`文件同名的`pxd`文件时，在`pxd`文件中定义的`cdef`函数将能在其它的Cython模块中调用。

In [2]:
%%file funclib.pxd

cdef double add(double a, double b)

Writing funclib.pxd


In [3]:
%%file funclib.pyx
from libc.stdint cimport uintptr_t

cdef double add(double a, double b):
    return a + b

def address_of_add():
    return <uintptr_t> add

Writing funclib.pyx


调用`cythonize`系统命令编译`funclib`模块：

In [4]:
!cythonize -b funclib.pyx

Compiling C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython\funclib.pyx because it changed.
[1/1] Cythonizing C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython\funclib.pyx
running build_ext
building 'funclib' extension
creating build
creating build\temp.win-amd64-3.5
creating build\temp.win-amd64-3.5\Release
creating build\temp.win-amd64-3.5\Release\Users
creating build\temp.win-amd64-3.5\Release\Users\RY
creating build\temp.win-amd64-3.5\Release\Users\RY\Documents
creating build\temp.win-amd64-3.5\Release\Users\RY\Documents\notebooks
creating build\temp.win-amd64-3.5\Release\Users\RY\Documents\notebooks\cooknotebook
creating build\temp.win-amd64-3.5\Release\Users\RY\Documents\notebooks\cooknotebook\notebooks
creating build\temp.win-amd64-3.5\Release\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython
C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Anaconda3\include -IC:\Anaconda3\includ

Cython模块通过`__pyx_capi__`输出`pxd`中定义的`cdef`函数：

In [5]:
import funclib
funclib.__pyx_capi__

{'add': <capsule object "double (double, double)" at 0x000001FE4A02E9C0>}

`PyCapsule`对象用于包装C语言的指针：

In [6]:
add_capsule = funclib.__pyx_capi__["add"]
type(add_capsule)

PyCapsule

## 调用Python API查看函数的地址

可以通过`ctypes`模块中提供的Python API函数获取`PyCapsule`对象包装的指针的值。下面调用`PyCapsule_GetName()`获取`PyCapsule`对象的名字，然后调用`PyCapsule_GetPointer()`获取其包装的指针，并与`address_of_add()`的返回值比较：

In [7]:
import ctypes
from ctypes import pythonapi

pythonapi.PyCapsule_GetName.restype = ctypes.c_char_p
pythonapi.PyCapsule_GetName.argtypes = [ctypes.py_object]

pythonapi.PyCapsule_GetPointer.restype = ctypes.c_void_p
pythonapi.PyCapsule_GetPointer.argtypes = [ctypes.py_object, ctypes.c_char_p]

name = pythonapi.PyCapsule_GetName(add_capsule)
addr = pythonapi.PyCapsule_GetPointer(add_capsule, name)
addr == funclib.address_of_add()

True

## 使用cimport从pxd文件中载入函数

下面的`funcall.pyx`模块从`funclib.pxd`载入`add`：

In [8]:
%%file funcall.pyx
from libc.stdint cimport uintptr_t
from funclib cimport add

def add_list(list a, list b):
    cdef list r
    r = []
    for v1, v2 in zip(a, b):
        r.append(add(v1, v2))
    return r

def address_of_add():
    return <uintptr_t>add

Writing funcall.pyx


In [9]:
!cythonize -b funcall.pyx

Compiling C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython\funcall.pyx because it changed.
[1/1] Cythonizing C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython\funcall.pyx
running build_ext
building 'funcall' extension
C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Anaconda3\include -IC:\Anaconda3\include "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\INCLUDE" "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\ATLMFC\INCLUDE" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.15063.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\shared" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\um" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\winrt" /TcC:\Users\RY\Documents\notebooks\cooknotebook\notebooks\cython\funcall.c /Fobuild\temp.win-amd64-3.5\Release\Users\RY\Documents\notebooks\cooknote

下面比较`funcall`和`funclib`模块中的`add()`函数的地址，它们的地址相同。如果单独载入`funcall`模块，它会自动载入`funclib`模块，并使用其中的`__pyx_capi__`字典初始化指向其中的函数的全局函数指针变量。这段初始化程序可以在`funcall.c`中找到。

In [10]:
import funcall
funcall.address_of_add() == funclib.address_of_add()

True

## 在Cython中调用BLAS和LAPACK函数

在SciPy 0.16中，使用上述方法提供了BLAS和LAPACK函数库，可以在Cython中调用其中的函数：

In [13]:
%%cython
from libc.stdint cimport uintptr_t
from scipy.linalg.cython_blas cimport dasum

def abs_sum(double[:] a):
    cdef int length, step
    length = a.shape[0]
    step = a.strides[0] // sizeof(double)
    return dasum(&length, &a[0], &step)

def addressof_dasum():
    return <uintptr_t>dasum

In [15]:
import numpy as np

a = np.random.rand(10)

print(abs_sum(a), np.sum(np.abs(a)))
print(abs_sum(a[::2]), np.sum(np.abs(a[::2])))

5.212678392237983 5.21267839224
1.987684796015655 1.98768479602


下面显示`cython_blas`中的所有函数。如果读者需要提高计算速度，可以参考[BLAS的文档](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-routines)，在Cython中调用这些函数。

In [22]:
from scipy.linalg import cython_blas as blas

print(list(blas.__pyx_capi__.keys()))

['zdscal', 'strsm', 'ssyr', 'strsv', 'dcabs1', 'strmv', 'ctpmv', 'ccopy', 'zgemv', 'sscal', 'stbmv', 'zsymm', 'stpmv', 'chpr', 'sger', 'ssyr2k', 'zdotu', 'dtrmm', 'cdotc', 'dswap', 'dsyr2', 'chpr2', 'dspr2', 'zhpmv', 'csyr2k', 'dtbmv', 'srot', 'zhpr', 'drotm', 'dtrsm', 'chemv', 'dtpmv', 'dspmv', 'ctrsv', 'srotg', 'drotg', 'csymm', 'stpsv', 'srotmg', 'sspr', 'sasum', 'ssymv', 'zher2k', 'ctpsv', 'dsyr2k', 'cswap', 'zgeru', 'zher2', 'zswap', 'sspmv', 'dgemv', 'zgemm', 'zsyr2k', 'cher', 'csrot', 'cgbmv', 'cgemm', 'dcopy', 'zgbmv', 'sspr2', 'ctrsm', 'sgemv', 'zdrot', 'icamax', 'saxpy', 'ztrsv', 'zher', 'ssymm', 'ssbmv', 'drotmg', 'zhbmv', 'cscal', 'drot', 'dtpsv', 'stbsv', 'cgeru', 'ctbmv', 'zsyrk', 'ztbsv', 'sswap', 'ztrsm', 'daxpy', 'dtrsv', 'dsdot', 'ztpmv', 'ztbmv', 'dtbsv', 'ddot', 'dscal', 'ztrmm', 'dger', 'dsbmv', 'lsame', 'zgerc', 'zhemv', 'ctbsv', 'zrotg', 'isamax', 'ssyrk', 'ctrmv', 'chbmv', 'idamax', 'cgerc', 'chpmv', 'sdot', 'cgemv', 'cher2', 'chemm', 'dgemm', 'ctrmm', 'dtrmv', 